# HiClass Policies Comparison

In [1]:
import os
os.chdir("/home/app/src")

In [2]:
import time
import pandas as pd
import numpy as np
import text_normalizer
import hiclass.metrics
from sklearn import metrics
from scripts.build_df import build_df
from scripts.decode_id import decode_id
from sklearn.model_selection import train_test_split
from os import cpu_count
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from hiclass import LocalClassifierPerParentNode

[nltk_data] Downloading package stopwords to /home/app/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/app/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


#### Base df and auxiliary functions

In [3]:
def normalization(input):
    output = text_normalizer.normalize_corpus(
        input,
        html_stripping=True,
        contraction_expansion=True,
        accented_char_removal=True,
        text_lower_case=True,
        text_stemming=True,
        text_lemmatization=False,
        special_char_removal=True,
        remove_digits=False,
        stopword_removal=True,
        stopwords=text_normalizer.stopword_list
    )       
    return output

In [4]:
# Auxiliary function for creating new columns
def assign_precision(x):
    return hiclass.metrics.precision(x[0:1], x[1:2])
def assign_recall(x):
    return hiclass.metrics.recall(x[0:1], x[1:2])
def assign_f1(x):
    # Condition to avoid ZeroDivisionError
    if x["precision"] * x["recall"] == 0.0:
        return 0.0
    else:
        return 2 * x["precision"] * x["recall"] / (x["precision"] + x["recall"])

In [5]:
# Base dataframe
no_threshold_df = build_df(json_path='data/products.json', threshold=0) 

# Train test split
X = no_threshold_df['name'].copy()
X = normalization(X.apply(str))
y = no_threshold_df['path'].copy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20, 
    random_state=42,
)

# Initialize vectorizer
tfid_vectorizer = TfidfVectorizer(ngram_range=(1, 2)) 
X_train = tfid_vectorizer.fit_transform(X_train)
X_test = tfid_vectorizer.transform(X_test)

In [6]:
train_comparison_df = pd.DataFrame(columns=["train_failures", "train_precision", "train_recall", "train_f1", "training_time"])
test_comparison_df = pd.DataFrame(columns=["test_failures", "test_precision", "test_recall", "test_f1", "training_time"])

# Model training

In [7]:
classifiers = [
    # LogisticRegression(max_iter=1000, random_state=42), 
    # RandomForestClassifier(random_state=42), 
    # DecisionTreeClassifier(random_state=42), 
    LGBMClassifier(reg_lambda = 0.4, n_estimators=100, n_jobs=-1, random_state=42), 
    ]

In [8]:
for classifier in classifiers:
    # Initialize timer
    starting_time = time.time()
    
    # Initialize hierarchical classifier with model
    hierarchical_classifier = hiclass.LocalClassifierPerParentNode(local_classifier=classifier, n_jobs=cpu_count())
    # Train hierarchical classifier
    hierarchical_classifier.fit(X_train, y_train)
    
    # Store training time
    training_time = time.time() - starting_time
    training_time = time.strftime("%H:%M:%S", time.gmtime(training_time))

    # Predict on train and test
    y_pred_train = hierarchical_classifier.predict(X_train)
    y_pred_test = hierarchical_classifier.predict(X_test)

    ## Save results
    # Train
    train_df = pd.DataFrame(y_train)
    train_df["prediction"] = pd.Series(y_pred_train.tolist(), index = train_df.index)
    train_df["precision"] = train_df.apply(lambda x: assign_precision(x), axis=1)
    train_df["recall"] = train_df.apply(lambda x: assign_recall(x), axis=1)
    train_df["f1"] = train_df.apply(lambda x: assign_f1(x), axis=1)
    
    results_train = [
        len(train_df["f1"][train_df["f1"] == 0.0]),
        hiclass.metrics.precision(y_train, y_pred_train),
        hiclass.metrics.recall(y_train, y_pred_train),
        hiclass.metrics.f1(y_train, y_pred_train),
        training_time
        ]

    train_comparison_df.loc[len(train_comparison_df)] = results_train
    
    # Test
    test_df = pd.DataFrame(y_test)
    test_df["prediction"] = pd.Series(y_pred_test.tolist(), index = test_df.index)
    test_df["precision"] = test_df.apply(lambda x: assign_precision(x), axis=1)
    test_df["recall"] = test_df.apply(lambda x: assign_recall(x), axis=1)
    test_df["f1"] = test_df.apply(lambda x: assign_f1(x), axis=1)

    results_test = [
        len(test_df["f1"][test_df["f1"] == 0.0]),
        hiclass.metrics.precision(y_test, y_pred_test),
        hiclass.metrics.recall(y_test, y_pred_test),
        hiclass.metrics.f1(y_test, y_pred_test),
        training_time
        ]

    test_comparison_df.loc[len(test_comparison_df)] = results_test
    
    print(training_time)

00:03:51


In [9]:
rename_dict = {
    0 : "LogisticRegression",
    1 : "RandomForestClassifier",
    2 : "DecisionTreeClassifier",
    3 : "LGBMClassifier",
    4 : "LGBMClassifier_tunned",
    5 : "LGBMClassifier_multiclass",
    6 : "LGBMClassifier_3",
    7 : "LGBMClassifier"
}

In [12]:
train_comparison_df.rename(index=rename_dict)

,train_failures,train_precision,train_recall,train_f1,training_time
LogisticRegression,559,0.901844,0.964525,0.932132,00:03:51


In [11]:
test_comparison_df.rename(index=rename_dict)

,test_failures,test_precision,test_recall,test_f1,training_time
LogisticRegression,580,0.802806,0.860049,0.830442,00:03:51
